In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from datetime import datetime
import pubmed as pm
import pandas as pd
import polars as pl
import utils
import time
import os

In [22]:
path = utils.path()
log_path = f'{path.split("/PDBs_releases")[0]}/log'
url = 'http://webs.iiitd.edu.in/raghava/cancerppd/browse_peptide.php'

In [23]:
data = {
    'CancerPPD ID': [],
    'PMID': [],
    'YEAR': [],
    'SEQUENCE': [],
    'NAME': [],
    'LENGTH': [],
    'LINEAR/CYCLIC': [],
    'CHIRAL': [],
    'CHEM-MOD': [],
    'C-ter MOD': [],
    'N-ter MOD': [],
    'NATURE': [],
    'ORIGIN': [],
    'CELL LINE': [],
    'CANCER TYPE': [],
    'ASSAY': [],
    'ACTIVITY': [],
    'TESTING TIME': [],
    'TISSUE EFFECTED': [],
    'PATENTS': [],
    'SMILES': []
}

In [27]:
driver = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 109
Current browser version is 111.0.5563.64 with binary path /usr/bin/google-chrome
Stacktrace:
#0 0x557fc5399303 <unknown>
#1 0x557fc516dd37 <unknown>
#2 0x557fc519aa54 <unknown>
#3 0x557fc519580e <unknown>
#4 0x557fc5192330 <unknown>
#5 0x557fc51d34a6 <unknown>
#6 0x557fc51ca753 <unknown>
#7 0x557fc519da14 <unknown>
#8 0x557fc519eb7e <unknown>
#9 0x557fc53e832e <unknown>
#10 0x557fc53ebc0e <unknown>
#11 0x557fc53ce610 <unknown>
#12 0x557fc53ecc23 <unknown>
#13 0x557fc53c0545 <unknown>
#14 0x557fc540d6a8 <unknown>
#15 0x557fc540d836 <unknown>
#16 0x557fc5428d13 <unknown>
#17 0x7ff0868c5609 start_thread


In [5]:
count_errors = []
error_line = ''
count = 1 # max=623
while True:
    driver.get(url)
    time.sleep(2)
    
    #Show all peptides
    select = Select(driver.find_element(By.XPATH, '/html/body/table[4]/tfoot/tr/td/select'))
    select.select_by_value('1000')
    
    #Show details from one peptide
    try:
        pep = driver.find_element(By.XPATH, f'/html/body/table[4]/tbody/tr[{count}]/td[1]/a')
        error_line = f'{count}, {pep.text}'
        pep.click()
        time.sleep(2)
    except:
        print('Finished')
        break
        
    #Capture data
    try:
        table = driver.find_element(By.XPATH, '/html/body/div[4]/div[1]/table/tbody')
        elements = table.find_elements(By.TAG_NAME, 'tr')
        for element in elements:
            props = element.find_elements(By.TAG_NAME, 'td')
            index = 0
            for key in data.keys():
                data[key].append(props[index].text)
                index += 1
    except:
        count_errors.append(error_line)
    finally:
        count += 1

Finished


In [34]:
# Create DataFrame
df = pd.DataFrame(data)

In [37]:
# Get DOI from PubMed
doi_list = []

#for pmid in data['PMID']:
for pmid in df['PMID']:
    if pmid != 'None':
        if len(pmid.split(',')) > 1:
            [pmid1, pmid2] = pmid.split(', ')
            doi1 = pm.get_doi_from_pmid(driver, pmid1)
            doi2 = pm.get_doi_from_pmid(driver, pmid2)
            doi_list.append([doi1, doi2])
        else:
            doi = pm.get_doi_from_pmid(driver, pmid)
            doi_list.append(doi)
    else:
        doi_list.append('Not Found')
    
df['DOI'] = doi_list

In [26]:
driver.close()

NameError: name 'driver' is not defined

In [26]:
#Write log file with failed peptides download 
dt = datetime.now()
with open(f'{log_path}/CancerPPD_errors_{dt.strftime("%Y_%m_%d_%H_%M")}.log', 'w', encoding='utf8') as file:
    print('failed peptides download:', len(count_errors))
    for value in count_errors:
        file.write(value)
        print(value.split(', '))

failed peptides download: 13
['126', 'FALALKALKK']
['335', 'I-C28H46N4O6,II-C27H44N4O6,III-C26H42N4O6']
['376', 'KKβ2,2WKK']
['474', 'KWβ2,2KK']
['516', 'NotAvailable']
['616', 'YHWYGYTPQNVIGGGKLlLKlLkkLLKlLKKK']
['617', 'YKQCHKKGGHCFPKEKICLPPSSDFGKMDCRWRWKCCKKGSG']
['618', 'YPFPG']
['619', 'YRA*G']
['620', 'YRWYGYTPQNVIGGGKLlLKlLkkLLKlLKKK']
['621', 'YSA*']
['622', 'YSA*G']
['623', 'YSL']


In [40]:
# Save csv file
utils.pd_to_csv(df, path, 'CancerPPD')

In [9]:
df